In [8]:
try :
    import stable_baselines3
except :
    ! pip install stable_baselines3
    
try :
    import shimmy
except:
    ! pip install shimmy

  Obtaining dependency information for stable_baselines3 from https://files.pythonhosted.org/packages/1e/43/d4b83e644c7e42d90d76a1987fb98a2ab286a2b5593350210ca8efcc378e/stable_baselines3-2.2.1-py3-none-any.whl.metadata
  Using cached stable_baselines3-2.2.1-py3-none-any.whl.metadata (5.0 kB)
Using cached stable_baselines3-2.2.1-py3-none-any.whl (181 kB)


In [2]:
! pip install --upgrade gym

^C
ERROR: Operation cancelled by user


In [4]:
import numpy as np
if int(np.__version__.split(".")[1]) >= 22:
    ! pip uninstall numpy --y
    ! pip install numpy==1.22.0
    
import numpy as np

Found existing installation: numpy 1.26.1
Uninstalling numpy-1.26.1:
  Successfully uninstalled numpy-1.26.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 68.3 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
chex 0.1.84 requires numpy>=1.24.1, but you have numpy 1.22.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.3 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, 

# Snake game logic

In [12]:
import random
import numpy as np

N = 10

class SnakeGame:
    def __init__(self, size=N):
        self.size = size
        self.reset()
        self.raw_obs = self.get_raw_observation()

    def reset(self):
        self.snake = [(self.size // 2, self.size // 2)]
        self.score = 0
        self.food = None
        self._place_food()
        self.game_over = False

    def _place_food(self):
        while self.food is None or self.food in self.snake:
            self.food = (random.randint(0, self.size - 1), random.randint(0, self.size - 1))

    def step(self, action):
        if self.game_over:
            return self.raw_obs, self.score, self.game_over, {}

        # Directions: 0-Up, 1-Down, 2-Right,  3-Left
        direction = [(0, -1), (0, 1), (1, 0),  (-1, 0)][action]
        new_head = (self.snake[0][0] + direction[0], self.snake[0][1] + direction[1])

        # Check for game over conditions
        if (new_head in self.snake) or new_head[0] < 0 or new_head[0] >= self.size or new_head[1] < 0 or new_head[1] >= self.size:
            self.game_over = True
            return self.raw_obs, self.score, self.game_over, {}

        self.snake.insert(0, new_head)

        # Check if snake eats food
        if new_head == self.food:
            self.score += 1
            self._place_food()
        else:
            self.snake.pop()

        self.raw_obs = self.get_raw_observation()
        return self.raw_obs, self.score, self.game_over, {}

    def get_raw_observation(self):
        raw_obs = np.zeros((self.size, self.size))
        for x, y in self.snake:
            raw_obs[y,x] = 1
        x, y = self.food
        raw_obs[y,x] = 2
        return raw_obs
    
    def render(self):
        raw_obs = self.get_raw_observation()
        for line in raw_obs :
            print(line, end="\n")
        print()
        
    def quit(self):
        pass

# Gym env 

In [162]:
# import gymnasium as gym
import gym
import numpy as np

MAX_LENGHT = 10

class SnakeEnv(gym.Env):
    """
    step(action): This method takes an action as input, updates the game state based on that action, returns the new state, the reward gained (or lost), whether the game is over (done), and additional info if necessary.
    reset(): This method resets the environment to an initial state and returns this initial state. It's used at the beginning of a new episode.
    render(): This method is for visualizing the state of the environment. Depending on how you want to view the game, this could simply update the game window.
    close(): This method performs any necessary cleanup, like closing the game window.
    """
    
    def __init__(self):
        super(SnakeEnv, self).__init__()
        self.action_space = gym.spaces.Discrete(4) # Output
        self.observation_space = gym.spaces.Box(low=-N, high=N,
                                            shape=(13,), dtype=np.float64)
        self.snake_game = None
        self.previous_score = 0
        self._last_distance = 0
        # StableBaselines throws error if these are not defined
        self.spec = None
        self.metadata = {"renders_mode":["human"]}
        
    def seed(self, seed=42): # needed with make_vec_env
        np.random.seed(seed)
    
    def get_snake_and_food_position(self, raw_obs):
        food_position = np.argwhere(raw_obs == 2).flatten()
        snake_positions = np.argwhere(raw_obs == 1).flatten()
        return snake_positions, food_position
        
    def euclidean_distance_centroid(self, raw_obs):
        snake_positions, food_position = self.get_snake_and_food_position(raw_obs)
        snake_centroid = np.mean(snake_positions)

        new_distance = np.linalg.norm(snake_centroid - food_position)
        return new_distance
    
    def feature_gen(self, raw_obs):
        new_distance = self.euclidean_distance_centroid(raw_obs)
        delta_distance = np.array([self._last_distance - new_distance])
        
        snake_positions, food_positions = self.get_snake_and_food_position(raw_obs)
        delta_lenght = max(MAX_LENGHT - len(snake_positions), 0)
        snake_positions = np.pad(snake_positions, (0, delta_lenght), mode='constant')

        obs = np.concatenate([snake_positions.flatten(), food_positions.flatten(), delta_distance])
        return obs
        
    def get_reward(self, raw_obs, score, done):
        # Calculate the Euclidean distance between the snake and the food
        new_distance = self.euclidean_distance_centroid(raw_obs)
        # Check if the snake has eaten food and update the reward
        if self.previous_score != score:
            reward = 100
            self.previous_score = score
        elif done:
            reward = -10
        else:
            reward =  1/2 if new_distance < self._last_distance else -1/10
        self._last_distance = new_distance
        return reward

    def step(self, action):
        raw_obs, score, done, _ = self.snake_game.step(action)
        obs = self.feature_gen(raw_obs)
        reward = self.get_reward(raw_obs, score, done)
        return obs, reward, done, _

    def reset(self):
        self.snake_game = SnakeGame()
        raw_obs = self.snake_game.raw_obs
        obs = self.feature_gen(raw_obs)
        return obs

    def render(self, mode='human'):
        if mode == 'human':
            self.snake_game.render()
            
    def close(self):
        self.snake_game.quit()


In [163]:
obs = SnakeEnv().reset()
obs.shape

(13,)

In [69]:
from stable_baselines3.common.env_checker import check_env

env = SnakeEnv()
# check_env(env)

# Reinforcement learning

In [139]:

import torch
import torch.nn as nn
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

class LinearQNet(BaseFeaturesExtractor):

    def __init__(self, observation_space, features_dim=32):
        super(LinearQNet, self).__init__(observation_space, features_dim)
        self.flatten = nn.Flatten()
        
        with torch.no_grad():
            n_flatten = self.flatten(
                torch.as_tensor(observation_space.sample()[None]).float()
            ).shape[1]
        
        self.linear = nn.Sequential(
                nn.Linear(n_flatten, features_dim),
                nn.ReLU(),
                nn.Linear(features_dim, features_dim),
                nn.ReLU(),
                nn.Linear(features_dim, features_dim),
                nn.ReLU(),
                nn.Linear(features_dim, features_dim),
                nn.ReLU(),
        )

    def forward(self, X):
        flat = self.flatten(X)
        out = self.linear(flat)
        return out

    
def evaluate_model(model, eval_env, num_episodes=10):
    all_rewards = []
    for episode in range(num_episodes):
        print(f"evaluation {episode=}")
        obs = eval_env.reset()
        done = False
        total_rewards = 0
        for _ in range(1000):
            action, _states = model.predict(obs, deterministic=True)
            obs, reward, done, _ = eval_env.step(action)
            total_rewards += reward
            if done :
                break
            
        all_rewards.append(total_rewards)
    average_reward = sum(all_rewards) / num_episodes
    return average_reward


# Training

The make_vec_env function from Stable Baselines 3 is used to create vectorized environments. <br>
Vectorized environments allow you to run multiple instances of an environment in parallel, <br>
providing a more efficient way to collect experiences (states, actions, rewards, etc.) during training.


In [164]:
%%time   
from stable_baselines3 import PPO,DQN, A2C
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.logger import configure
from stable_baselines3.common.utils import get_schedule_fn

train_env = make_vec_env(lambda: SnakeEnv(), n_envs=5)
eval_env = SnakeEnv()


policy_kwargs = dict(features_extractor_class=LinearQNet)

learning_rate_schedule = get_schedule_fn(0.000001)
model = PPO("MlpPolicy", train_env, policy_kwargs=policy_kwargs, verbose=2,learning_rate=learning_rate_schedule)               

new_logger = configure("path_to_save_logs", ["stdout", "tensorboard"])
model.set_logger(new_logger) # Run TensorBoard in a terminal: tensorboard --logdir=path_to_save_logs

total_timesteps = 1_000_000
eval_interval = 1_000_000  
num_eval_episodes = 10

# Training loop with periodic evaluation
for i in range(0, total_timesteps, eval_interval):
    print(f"Training session N°{i/eval_interval} {'-'*50}")
    model.learn(total_timesteps=eval_interval)
    avg_reward = evaluate_model(model, eval_env, num_episodes=num_eval_episodes)
    print(f"Evaluation average reward: {avg_reward}")

model.save("ppo_snake")

Using cuda device
Logging to path_to_save_logs
Training session N°0.0 --------------------------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 30.4     |
|    ep_rew_mean     | 42.1     |
| time/              |          |
|    fps             | 1695     |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 10240    |
---------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 33.9          |
|    ep_rew_mean          | 24.7          |
| time/                   |               |
|    fps                  | 983           |
|    iterations           | 2             |
|    time_elapsed         | 20            |
|    total_timesteps      | 20480         |
| train/                  |               |
|    approx_kl            | 3.5808014e-06 |
|    clip_fraction        | 0             |
|    clip_r

# Tensorboard

In [ ]:
! tensorboard --logdir=path_to_save_logs

# Testing

In [ ]:
from stable_baselines3 import PPO 


env = SnakeEnv()
model = PPO.load("ppo_snake", env=env)

obs = env.reset()
done = False
env.render()
while not done:
    #input("press enter to continue")
    action, _info = model.predict(obs, deterministic=True)
    obs, reward, done, _ = env.step(action)
    env.render()